In [34]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import random
import rasterio
from PIL import Image

import requests
import zipfile
from pathlib import Path

from matplotlib import pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets

# Note: this notebook requires torch >= 1.10.0
torch.__version__

'2.0.0+cpu'

In [35]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [36]:
# Setup path to data folder
data_path = Path("../data/")
image_path = data_path / "Euro_MS_Test_red"

In [37]:
import os
def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.
  Args:
    dir_path (str or pathlib.Path): target directory
  
  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [38]:
walk_through_dir(image_path)

There are 2 directories and 0 images in '..\data\Euro_MS_Test_red'.
There are 10 directories and 0 images in '..\data\Euro_MS_Test_red\test'.
There are 0 directories and 6 images in '..\data\Euro_MS_Test_red\test\AnnualCrop'.
There are 0 directories and 6 images in '..\data\Euro_MS_Test_red\test\Forest'.
There are 0 directories and 6 images in '..\data\Euro_MS_Test_red\test\HerbaceousVegetation'.
There are 0 directories and 5 images in '..\data\Euro_MS_Test_red\test\Highway'.
There are 0 directories and 5 images in '..\data\Euro_MS_Test_red\test\Industrial'.
There are 0 directories and 4 images in '..\data\Euro_MS_Test_red\test\Pasture'.
There are 0 directories and 5 images in '..\data\Euro_MS_Test_red\test\PermanentCrop'.
There are 0 directories and 6 images in '..\data\Euro_MS_Test_red\test\Residential'.
There are 0 directories and 5 images in '..\data\Euro_MS_Test_red\test\River'.
There are 0 directories and 6 images in '..\data\Euro_MS_Test_red\test\SeaLake'.
There are 10 directori

In [39]:
# Setup train and testing paths
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(WindowsPath('../data/Euro_MS_Test_red/train'),
 WindowsPath('../data/Euro_MS_Test_red/test'))

In [40]:
# Write transform for image
data_transform = transforms.Compose([
    # Turn the image into a torch.Tensor
    transforms.ToTensor() # this also converts all pixel values from 0 to 255 to be between 0.0 and 1.0 
])

In [41]:
class CustomImageFolder_Size(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes, self.class_to_idx = self._find_classes()
        self.samples = self._make_dataset()

    def _find_classes(self):
        classes = [d.name for d in os.scandir(self.root_dir) if d.is_dir()]
        classes.sort()
        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        return classes, class_to_idx

    def _make_dataset(self):
        images = []
        for target_class in sorted(self.class_to_idx.keys()):
            class_index = self.class_to_idx[target_class]
            target_dir = os.path.join(self.root_dir, target_class)
            for root, _, fnames in sorted(os.walk(target_dir, followlinks=True)):     
                for fname in sorted(fnames):
                    path = os.path.join(root, fname)
                    item = (path, class_index)
                    images.append(item)
        return images

    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        path, target = self.samples[index]
        with rasterio.open(path) as f:
            img_dat = f.read()
            img = np.asarray(img_dat,dtype=np.float32)
        if self.transform is not None:
            img = self.transform(img)
            img = img.permute(1,0,2)                      # to get [height,width,colour_channel as op]
        return img, target
    
    def set_samples(self, num_samples):
        self.samples = self.samples[:num_samples]

In [42]:
# Use ImageFolder to create dataset(s)

# presently testing out a data set for customized size

train_data = CustomImageFolder_Size(root_dir=train_dir, # target folder of images
                                  transform=data_transform,
                                  ) # transforms to perform on data (images)
                        

test_data = CustomImageFolder_Size(root_dir=test_dir, 
                                 transform=data_transform,)

print(f"Train data:\n{train_data}\nTest data:\n{test_data}")

Train data:
Test data:


In [43]:
class_names = train_data.classes
class_names

['AnnualCrop',
 'Forest',
 'HerbaceousVegetation',
 'Highway',
 'Industrial',
 'Pasture',
 'PermanentCrop',
 'Residential',
 'River',
 'SeaLake']

In [44]:
# Can also get class names as a dict
class_dict = train_data.class_to_idx
class_dict

{'AnnualCrop': 0,
 'Forest': 1,
 'HerbaceousVegetation': 2,
 'Highway': 3,
 'Industrial': 4,
 'Pasture': 5,
 'PermanentCrop': 6,
 'Residential': 7,
 'River': 8,
 'SeaLake': 9}

In [45]:
# Turn train and test Datasets into DataLoaders
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset=train_data, 
                              batch_size=1, # how many samples per batch?
                              #num_workers=1, # how many subprocesses to use for data loading? (higher = more)
                              shuffle=True) # shuffle the data?

test_dataloader = DataLoader(dataset=test_data, 
                             batch_size=1, 
                             #num_workers=1, 
                             shuffle=False) # don't usually need to shuffle testing data

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x18381a503d0>,
 <torch.utils.data.dataloader.DataLoader at 0x18381a50430>)

In [46]:
img, label = next(iter(train_dataloader))

# Batch size will now be 1, try changing the batch_size parameter above and see what happens
print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label.shape}")

Image shape: torch.Size([1, 13, 64, 64]) -> [batch_size, color_channels, height, width]
Label shape: torch.Size([1])


In [47]:
class TinyVGG(nn.Module):
    """
    Model architecture copying TinyVGG from: 
    https://poloclub.github.io/cnn-explainer/
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                      out_channels=hidden_units, 
                      kernel_size=3, # how big is the square that's going over the image?
                      stride=1, # default
                      padding=1), # options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number 
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) # default stride value is same as kernel_size
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            # Where did this in_features shape come from? 
            # It's because each layer of our network compresses and changes the shape of our inputs data.
            nn.Linear(in_features=hidden_units*16*16,
                      out_features=output_shape)
        )
    
    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        # print(x.shape)
        x = self.conv_block_2(x)
        # print(x.shape)
        x = self.classifier(x)
        # print(x.shape)
        return x
        # return self.classifier(self.conv_block_2(self.conv_block_1(x))) # <- leverage the benefits of operator fusion



In [51]:
def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module):
    # Put model in eval mode
    model.eval() 
    
    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0
    
    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)
    
            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()
            
            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
            
    # Adjust metrics to get average loss and accuracy per batch 
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

In [57]:
model_0 = torch.load('../models/EuroSAT_M1_H39')

In [58]:
try: 
    import torchinfo
except:
    !pip install torchinfo
    import torchinfo
    
from torchinfo import summary
summary(model_0, input_size=[1, 13, 64, 64]) 

Layer (type:depth-idx)                   Output Shape              Param #
TinyVGG                                  [1, 10]                   --
├─Sequential: 1-1                        [1, 39, 32, 32]           --
│    └─Conv2d: 2-1                       [1, 39, 64, 64]           4,602
│    └─ReLU: 2-2                         [1, 39, 64, 64]           --
│    └─Conv2d: 2-3                       [1, 39, 64, 64]           13,728
│    └─ReLU: 2-4                         [1, 39, 64, 64]           --
│    └─MaxPool2d: 2-5                    [1, 39, 32, 32]           --
├─Sequential: 1-2                        [1, 39, 16, 16]           --
│    └─Conv2d: 2-6                       [1, 39, 32, 32]           13,728
│    └─ReLU: 2-7                         [1, 39, 32, 32]           --
│    └─Conv2d: 2-8                       [1, 39, 32, 32]           13,728
│    └─ReLU: 2-9                         [1, 39, 32, 32]           --
│    └─MaxPool2d: 2-10                   [1, 39, 16, 16]           --


In [59]:
loss_fn = nn.CrossEntropyLoss()
test_l, test_ac = test_step(model=model_0,
          dataloader = test_dataloader,
          loss_fn = loss_fn)

print("Test loss : ",test_l," | Test accuracy : ",test_ac)

Test loss :  0.009372412825861695  | Test accuracy :  1.0
